# Compute Power Transfer Distribution Factors

In [24]:
import polars as pl
from benchmark_utils import mock_snakemake
from scipy.sparse import coo_array, diags_array
from sksparse.cholmod import cholesky

DF_CUTOFF = 1e-10

if "snakemake" not in globals():  # noqa: F821
    snakemake = mock_snakemake("compute_ptdf")

lines = pl.read_parquet(
    snakemake.input.lines, columns=["line_id", "from_bus", "to_bus", "reactance"]
)
L = lines.height

In [25]:
# Step 1: Renumber lines from 1 to L contiguously.
lines = lines.sort("line_id")
lines = lines.with_row_index(name="line_index")
lines_map = lines.select("line_id", "line_index")
lines = lines.drop("line_id")
lines

line_index,from_bus,to_bus,reactance
u32,i64,i64,f64
0,1,2410,0.000219
1,20,2410,0.007485
2,2410,2411,0.016138
3,1618,2411,0.000469
4,1617,2411,0.004104
…,…,…,…
8119,8854,8855,0.115871
8120,8861,8862,0.034818
8121,8860,8862,0.020603


In [26]:
# Step 2: Renumber buses such that they go from 1 to N contiguously.
bus_map = (
    pl.concat([lines["from_bus"], lines["to_bus"]])
    .unique()
    .sort()
    .rename("bus")
    .to_frame()
    .with_row_index(name="bus_index")
)
N = bus_map.height

lines = (
    lines.join(bus_map, left_on="from_bus", right_on="bus", coalesce=True)
    .drop("from_bus")
    .rename({"bus_index": "from_bus"})
    .join(bus_map, left_on="to_bus", right_on="bus", coalesce=True)
    .drop("to_bus")
    .rename({"bus_index": "to_bus"})
)
lines

line_index,reactance,from_bus,to_bus
u32,f64,u32,u32
0,0.000219,0,1982
1,0.007485,17,1982
2,0.016138,1982,1983
3,0.000469,1322,1983
4,0.004104,1321,1983
…,…,…,…
8119,0.115871,6461,6462
8120,0.034818,6465,6466
8121,0.020603,6464,6466


In [27]:
# Step 3: Form an admittance matrix in COO form (row_index (i), col_index (j), value)
lines = lines.with_columns(suscep=1 / lines["reactance"])
off_diagonal = pl.concat(
    [
        lines.select(
            pl.col("from_bus").alias("i"),
            pl.col("to_bus").alias("j"),
            -pl.col("suscep").alias("val"),
        ),
        lines.select(
            pl.col("to_bus").alias("i"),
            pl.col("from_bus").alias("j"),
            -pl.col("suscep").alias("val"),
        ),
    ]
)

diagonal = (
    pl.concat(
        [
            lines.select("suscep", i=pl.col("from_bus")),
            lines.select("suscep", i=pl.col("to_bus")),
        ]
    )
    .group_by("i")
    .agg(pl.sum("suscep").alias("val"))
    .select("i", pl.col("i").alias("j"), "val")
)
assert len(diagonal) == N, "Diagonal entries should match the number of buses."
admittance_matrix_df = pl.concat([off_diagonal, diagonal])
admittance_matrix_df

i,j,val
u32,u32,f64
0,1982,-4566.772343
17,1982,-133.59381
1982,1983,-61.96631
1322,1983,-2133.621717
1321,1983,-243.643906
…,…,…
1441,1441,525.29819
2358,2358,4141.423826
4454,4454,156.421335


In [28]:
# Step 4: Drop last row and column (slack bus).
admittance_matrix_df = admittance_matrix_df.filter(
    pl.col("i") < (N - 1), pl.col("j") < (N - 1)
)
admittance_matrix = coo_array(
    (
        admittance_matrix_df["val"].to_numpy(),
        (
            admittance_matrix_df["i"].cast(pl.Float64).to_numpy(),
            admittance_matrix_df["j"].cast(pl.Float64).to_numpy(),
        ),
    ),
    shape=(N - 1, N - 1),
)
admittance_matrix

<COOrdinate sparse array of dtype 'float64'
	with 22714 stored elements and shape (6472, 6472)>

In [29]:
# Step 5: Inverse the admittance matrix to get voltage angles.
# We use the sparse Cholesky factorization since, based on my tests, this is much faster than using any of the following:
# scipy.linalg.inv, scipy.sparse.linalg.inv, scipy.linag.pinv, scipy.sparse.linalg.spsolve
factor = cholesky(admittance_matrix.tocsc())
voltage_angles = factor.inv()
voltage_angles

/tmp/ipykernel_2203686/2315113477.py:5: CholmodTypeConversionWarning: array contains 32 bit integers; but 64 bit integers are needed; slowing down due to converting
  voltage_angles = factor.inv()


<Compressed Sparse Column sparse matrix of dtype 'float64'
	with 41886784 stored elements and shape (6472, 6472)>

In [31]:
# Step 6: Calculate the power flow by multiplying the voltage angles by diag(S)*A^T
# A^T is the L by N adjacency matrix
adjacency_matrix_T = pl.concat(
    [
        lines.select(
            i=pl.col("line_index"),
            j=pl.col("from_bus"),
            val=pl.lit(1),
        ).filter(pl.col("j") < N - 1),  # Exclude slack bus,
        lines.select(
            i=pl.col("line_index"),
            j=pl.col("to_bus"),
            val=pl.lit(-1),
        ).filter(pl.col("j") < N - 1),  # Exclude slack bus
    ]
)
adjacency_matrix_T = coo_array(
    (
        adjacency_matrix_T["val"].to_numpy(),
        (adjacency_matrix_T["i"].to_numpy(), adjacency_matrix_T["j"].to_numpy()),
    ),
    shape=(L, N - 1),
)
diag_suscep = diags_array(lines.sort("line_index")["suscep"].to_numpy())
power_flow = diag_suscep @ adjacency_matrix_T @ voltage_angles
power_flow = power_flow.tocoo()
power_flow_df = pl.DataFrame(
    {
        "injection": power_flow.col,
        "line_index": power_flow.row,
        "factor": power_flow.data,
    }
)
power_flow_df

injection,line_index,factor
i64,i64,f64
0,0,1.0
6471,1,-0.001003
6470,1,0.000805
6469,1,0.0008
6468,1,0.00078
…,…,…
4,8123,-0.000008
3,8123,0.000009
2,8123,-9.4857e-7


In [ ]:
# Step 7: Filter out small values
power_flow_df = power_flow_df.filter(pl.col("factor").abs() > DF_CUTOFF)
power_flow_df

injection,line_index,factor
i64,i64,f64
0,0,1.0
6471,1,-0.001003
6470,1,0.000805
6469,1,0.0008
6468,1,0.00078
…,…,…
4,8123,-0.000008
3,8123,0.000009
2,8123,-9.4857e-7


In [35]:
# Step 8: Unmap buses and lines to original IDs.
power_flow_df_unmapped = (
    power_flow_df.join(
        bus_map,
        left_on="injection",
        right_on="bus_index",
        coalesce=True,
        validate="m:1",
    )
    .drop("injection")
    .rename({"bus": "injection"})
    .join(lines_map, on="line_index")
    .drop("line_index")
    .select("injection", "line_id", "factor")
    .sort("injection", "line_id")
)
power_flow_df_unmapped

injection,line_id,factor
i64,i64,f64
1,1,1.0
1,2,-0.558747
1,3,0.441253
1,5,-0.441253
1,6,0.441253
…,…,…
8867,10815,-0.000006
8867,10818,-5.5862e-9
8867,10819,1.9651e-8


In [36]:
power_flow_df_unmapped.write_parquet(snakemake.output[0])
snakemake.output[0]

'/data/home/machstg/pyoframe/benchmarks/input_data/energy_planning/intermediary_data_inputs/power_transfer_dist_facts.parquet'